In [1]:
##### was created to test the implementation of the MASKed language model for recommendation systems 
#### Link of the research paper  BERT4Rec: Sequential Recommendation with Bidirectional Encoder Representations from Transformer
#### https://arxiv.org/pdf/1904.06690.pdf
#### as mentioned earlier masked language model is an natural language tast NLP
#### during the data preparation one token is replaced with a special token in our case 
#### this special token is ["mask"] . This situation is instanced below
#### training input sentence, " I liked this ['mask]"
#### target input, 'I liked this movie.'
#### At this point, It is better to explain the sentence structure, words can be have meaining individually in a sentence.
#### In addition, complete meaning of the sentence also depends of the ordering of the words.
#### It is not directly possible to apply the same procedure for recommender systems. 
#### In this case items will be sorted according to date as ascending order
####

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

import sys,os 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense, Embedding, LayerNormalization, MultiHeadAttention,Dropout, TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
SEQ_LEN = 64  ####  input length if the actual input is shorter than seq_len, rest will be zero padded
EMBED_DIM = 16 #### embedding dimension or latent dimension for the items vectors
CORPUS_SIZE = 30000 #### total number of unique items, this parameter also
                   #### will be used for the corpus size of the Embedding layer and the n_neurons for the softmax layer(decision layer)
N_NEURONS = 16 #### number of neuron for the sequential layer for feed forward neural network,
               #### to sum output of the attention layer and sequential layer N_NEURONS AND EMBED_DIM must be equal
NUM_HEADS = 8  #### number of heads for multi head attention layer
KEY_DIM = 16 #### key dimension parameter for the multihead attention 
NUM_ATT_LAYER = 1 #### number of encoder layer, 

In [4]:
dir_url = 'drive/MyDrive/Colab Notebooks/datasets/anime/animelists_cleaned.csv'
animelist_cleaned_cols = ['username','anime_id','my_last_updated'] ##since data set is large, 
                                                                       ##I will not add some non-related columns
df = pd.read_csv(dir_url,
                     usecols = animelist_cleaned_cols,
                    #  nrows = 100
                     )



In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31284030 entries, 0 to 31284029
Data columns (total 3 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   username         object
 1   anime_id         int64 
 2   my_last_updated  object
dtypes: int64(1), object(2)
memory usage: 716.0+ MB


In [6]:
df.head(3)

,username,anime_id,my_last_updated
0,karthiga,21,2013-03-03 10:52:53
1,karthiga,59,2013-03-10 13:54:51
2,karthiga,74,2013-04-27 16:43:35


In [7]:
def data_splitter(x):
  return x.split()[0]

df['my_last_updated'] = df['my_last_updated'].apply(data_splitter)


In [8]:

#### there are some entries which was created before year 2000, I will eliminate those entries.
var1 = np.where(df['my_last_updated'] <= '2000-00-00')

In [9]:
# list(var1[0])
df = df.drop(list(var1[0])).reset_index(drop=True)
df['my_last_updated'] = pd.to_datetime(df['my_last_updated'])
df.head(3)

,username,anime_id,my_last_updated
0,karthiga,21,2013-03-03
1,karthiga,59,2013-03-10
2,karthiga,74,2013-04-27


In [10]:
users = df['username'].unique()
animes = df['anime_id'].unique()
num_users = len(users)
num_animes = len(animes)

print(num_users)
print(num_animes)
mask_token_id = num_animes+1

108676
6668


In [11]:
anime_2enc = {i:j for i,j in enumerate(animes)}
enc_2anime = {j:i for i,j in enumerate(animes)}

user_2enc = {i:j for i,j in enumerate(users)}
enc_2users = {j:i for i,j in enumerate(users)}

In [12]:
encoded_animes = df['anime_id'].map(enc_2anime)
encoded_users = df['username'].map(enc_2users)

In [13]:
df_enc = pd.DataFrame({'username_enc':encoded_users, 'anime_enc':encoded_animes, 'my_last_updated':df['my_last_updated']})

In [14]:
df_enc.head(3)

,username_enc,anime_enc,my_last_updated
0,0,0,2013-03-03
1,0,1,2013-03-10
2,0,2,2013-04-27


In [15]:
#### users and anime_ids were sorted within the 'my_last_update' feature
#### animes were listed for each user from first date to last date 
x = df_enc.set_index(['username_enc','my_last_updated']).sort_index()
x = x.reset_index()

x.head(3)

,username_enc,my_last_updated,anime_enc
0,0,2013-03-03,0
1,0,2013-03-03,3
2,0,2013-03-03,9


In [16]:
x = x.drop(columns= ['my_last_updated'])  

In [17]:
x.head(3)

,username_enc,anime_enc
0,0,0
1,0,3
2,0,9


In [18]:
#### 
df_enc_seq = x.groupby('username_enc').aggregate(lambda tdf: tdf.unique().tolist())
df_enc_seq = df_enc_seq.reset_index()


In [19]:
df_enc_seq.head()

,username_enc,anime_enc
0,0,"[0, 3, 9, 12, 37, 52, 6, 34, 39, 40, 1, 5, 7, ..."
1,1,"[66, 78, 57, 55, 82, 204, 207, 213, 225, 184, ..."
2,2,"[1, 5, 6, 8, 11, 12, 18, 23, 24, 27, 36, 310, ..."
3,3,"[1, 55, 662, 64, 66, 72, 666, 671, 342, 348, 1..."
4,4,"[317, 66, 331, 84, 105, 112, 129, 999, 273, 27..."


In [20]:
min_len = 5
shorts = []
for i,j in enumerate(df_enc_seq['anime_enc']):
  if len(j) < min_len:
    shorts.append(i)

df_enc_seq = df_enc_seq.drop(shorts).reset_index(drop=True)

In [21]:
df_enc_seq.head(3)

,username_enc,anime_enc
0,0,"[0, 3, 9, 12, 37, 52, 6, 34, 39, 40, 1, 5, 7, ..."
1,1,"[66, 78, 57, 55, 82, 204, 207, 213, 225, 184, ..."
2,2,"[1, 5, 6, 8, 11, 12, 18, 23, 24, 27, 36, 310, ..."


In [22]:
lens = []
for i in df_enc_seq['anime_enc']:
  lens.append(len(i))

In [23]:
arranged_seqs = pad_sequences(
    df_enc_seq['anime_enc'], maxlen=SEQ_LEN, dtype='int32', padding='pre',
    truncating='pre', value=0.0
)


In [24]:
def get_masked_input_and_labels(encoded_texts):
    # 15% BERT masking
    inp_mask = np.random.rand(*encoded_texts.shape) < 0.15
    # Do not mask special tokens
    inp_mask[encoded_texts <= 2] = False
    # Set targets to -1 by default, it means ignore
    labels = -1 * np.ones(encoded_texts.shape, dtype=int)
    # Set labels for masked tokens
    labels[inp_mask] = encoded_texts[inp_mask]

    # Prepare input
    encoded_texts_masked = np.copy(encoded_texts)
    # Set input to [MASK] which is the last token for the 90% of tokens
    # This means leaving 10% unchanged
    inp_mask_2mask = inp_mask & (np.random.rand(*encoded_texts.shape) < 0.90)
    encoded_texts_masked[
        inp_mask_2mask
    ] = mask_token_id  # mask token is the last in the dict

    # Set 10% to a random token
    inp_mask_2random = inp_mask_2mask & (np.random.rand(*encoded_texts.shape) < 1 / 9)
    encoded_texts_masked[inp_mask_2random] = np.random.randint(
        3, mask_token_id, inp_mask_2random.sum()
    )

    # Prepare sample_weights to pass to .fit() method
    sample_weights = np.ones(labels.shape)
    sample_weights[labels == -1] = 0

    # y_labels would be same as encoded_texts i.e input tokens
    y_labels = np.copy(encoded_texts)

    return encoded_texts_masked, y_labels

In [25]:
x_input, y_input = get_masked_input_and_labels(arranged_seqs)

In [26]:
def get_pos_encoding_matrix(max_len, d_emb):
    pos_enc = np.array(
        [
            [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)]
            if pos != 0
            else np.zeros(d_emb)
            for pos in range(max_len)
        ]
    )
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
    return pos_enc

In [27]:
first_input = tf.keras.Input(shape = (SEQ_LEN,))
embedding_layer = Embedding(CORPUS_SIZE,EMBED_DIM)(first_input)
position_embeddings = layers.Embedding(
        input_dim=SEQ_LEN,
        output_dim=EMBED_DIM,
        weights=[get_pos_encoding_matrix(SEQ_LEN, EMBED_DIM)],
        name="position_embedding",
    )(tf.range(start=0, limit=SEQ_LEN, delta=1))
# total_embs = word_embeddings + get_pos_encoding_matrix(SEQUENCE_LEN,EMBED_DIM)
# total_embs = word_embeddings + position_embeddings
x = embedding_layer + position_embeddings
for i in range(NUM_ATT_LAYER):
  mha = MultiHeadAttention(num_heads = NUM_HEADS, key_dim = 16)(x,x,x)
  norm_1 = LayerNormalization(epsilon=1e-6)(mha+x)

  seq_model = tf.keras.Sequential([
                                   Dense(N_NEURONS, activation = 'relu'),
                                   Dense(EMBED_DIM)
  ])

  seq_layer = seq_model(norm_1)
  norm_2 =LayerNormalization(epsilon=1e-6)(seq_layer+norm_1)
  x = norm_2

top_layer = Dense(CORPUS_SIZE+1, activation='softmax', name = 'selection_layer')(x)
attention_model = tf.keras.Model(inputs = first_input, outputs = top_layer)


attention_model.compile(
    loss = 'SparseCategoricalCrossentropy',
    optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
      # beta_1=0.9, beta_2=0.999, epsilon=1e-07  
      ),
    metrics = ['accuracy'],
                      )
tf.keras.utils.plot_model(
    attention_model,
    to_file='model.png', show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB',
    expand_nested=False,
    dpi=68,
    layer_range=None, show_layer_activations=True
)

In [29]:
# print(x_input.shape)
# print(y_input.shape)
# train_data = tf.data.Dataset.from_tensor_slices((np.array(x_input), np.array(y_input)))
# x_inp_2 = x_input.reshape(1,x_input.shape[0],x_input.shape[1])
# y_inp_2 = y_input.reshape(1,y_input.shape[0],y_input.shape[1])
# train_data2 = tf.data.Dataset.from_tensor_slices((x_inp_2, y_inp_2))

from sklearn.model_selection import train_test_split
x_train,x_test, y_train,  y_test = train_test_split(x_input, y_input, train_size = 0.95,
                                                    test_size = 0.05, 
                                                    random_state = 42)

print('x_train',x_train.shape)
print('x_test',x_test.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)

x_train (101588, 64)
x_test (5347, 64)
y_train (101588, 64)
y_test (5347, 64)


In [30]:
# attention_hist = attention_model.fit(x_train,y_train, epochs = 10, batch_size = 250)


In [31]:
import os
# print(os.listdir('drive/MyDrive/Colab Notebooks'))
# attention_model.save("drive/MyDrive/Colab Notebooks/cloze_task.h5")
attention_model.load_weights('drive/MyDrive/Colab Notebooks/cloze_task.h5')



In [32]:
def mask_filler(model,mask_position, recommend_session, mask_token_id):
  # recommend  session : idx of the recommendation sequence in the test set 
  # mask position : position of the mask in the recommend session
  # y_train[recommend_session]
  y_test_check = y_test.copy()
  candidate_sequence = y_test_check[recommend_session]
  print(y_test[recommend_session])
  candidate_sequence[mask_position] = mask_token_id
  print(candidate_sequence)
  candidate_sequence = candidate_sequence.reshape(1, candidate_sequence.shape[0])
  mask_prediction = attention_model.predict([candidate_sequence])
  res = tf.math.top_k(mask_prediction[0][mask_position], k=20)
  return res[1]

In [33]:
mask_filler(attention_model,mask_position = 12, 
            recommend_session=7, mask_token_id=mask_token_id)

[ 995  269  578  736   75 3714  654 1010    0 1008  277 4184 2042 3766
 1462 1990   42  692 4183  243  740 2590  690 2018 1820  642   45  759
 2103  697  136 3969  842 2549 2581 1013 4036 2670 1030  698 1004 1795
  137 1020  811 1235 1848 3758  138  146 1045 1241   90 2166  992  272
  879  884  886  933 2197   81  112  141]
[ 995  269  578  736   75 3714  654 1010    0 1008  277 4184 6669 3766
 1462 1990   42  692 4183  243  740 2590  690 2018 1820  642   45  759
 2103  697  136 3969  842 2549 2581 1013 4036 2670 1030  698 1004 1795
  137 1020  811 1235 1848 3758  138  146 1045 1241   90 2166  992  272
  879  884  886  933 2197   81  112  141]


<tf.Tensor: shape=(20,), dtype=int32, numpy=
array([112,  19, 128, 690, 137,  66,   9, 691, 874,  27,  53, 702, 134,
        75, 149, 120, 130, 105, 159,  87], dtype=int32)>